In [5]:
import keras
import numpy as np
from keras import layers
import random
import sys

In [2]:
def reweight_distribution(original_distribution, temperature=2):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

In [3]:
ori_dstri = np.array([0.8, 0.1, 0.1])   # a、b、c 的機率分布

new_dstri = reweight_distribution(ori_dstri, temperature=0.01)  # 使用溫度 0.01 
print(new_dstri)    # [1.00000000e+00 4.90909347e-91 4.90909347e-91]

new_dstri = reweight_distribution(ori_dstri, temperature=2) # 使用預設溫度 2
print(new_dstri)    # [0.58578644 0.20710678 0.20710678]

new_dstri = reweight_distribution(ori_dstri, temperature=10) # 使用溫度 10
print(new_dstri)    # [0.38102426 0.30948787 0.30948787]

[1.00000000e+00 4.90909347e-91 4.90909347e-91]
[0.58578644 0.20710678 0.20710678]
[0.38102426 0.30948787 0.30948787]


In [6]:
path = keras.utils.get_file(        # 取得文本檔案
    'nietzsche.txt',                # 檔名 
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')  # 檔案位置
text = open(path, encoding='utf-8').read().lower()    # 讀取文本內容，並轉成小寫
print('Corpus length:', len(text))  # 文本長度為 600893

606208/600901 [==============================] - 1s 2us/step
Corpus length: 600893


In [7]:
maxlen = 60     # 每次 (step) 從文本中萃取 60 個字元作為序列資料
step = 3        # 每 3 個字元為一個 step 進行萃取 

sentences = []  # 存放萃取出的序列資料
next_chars = [] # 存放對應目標 (萃取出來的序列資料的後一個字元)

# 萃取序列資料 
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))   # 共萃取出 200278 個序列資料

Number of sequences: 200278


In [8]:
# 產生文本中的 "唯一" 字元串列 (文本轉成 set 將重複字元刪除)
chars = sorted(list(set(text))) 
print('Unique characters:', len(chars)) # 文本共使用 57 種字元

Unique characters: 57


In [9]:
# 將各個字元對應到 "chars" 串列中的索引值成為字典 (dict) 格式。即 {'\n': 0,' ': 1, '!': 2,…}
char_indices = dict((char, chars.index(char)) for char in chars)

# 將字元經 One-hot 編碼成二元陣列
print('Vectorization...')

Vectorization...


In [10]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print(x.shape)  # (200278, 60, 57)
print(y.shape)  # (200278, 57)

(200278, 60, 57)
(200278, 57)


In [11]:
model = keras.models.Sequential()   # 建立序列式模型
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [12]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [13]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature     # 重新加權計算 (熵)
    exp_preds = np.exp(preds)               
    preds = exp_preds / np.sum(exp_preds)   
    probas = np.random.multinomial(1, preds, 1) # 丟入多項式分布中
    return np.argmax(probas)

In [14]:
for epoch in range(1, 60):  # 共 60 個訓練週期 (次數)
    print('epoch', epoch)
    model.fit(x, y,         # 用萃取出來的 x, y 開始進行訓練
              batch_size=128,
              epochs=1)
    # 隨機選擇文本中的某段 60 個字元
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- 隨機初始文字: "' + generated_text + '"')

    # 嘗試使用一系列不同 temperature 生成文字
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # 每個 temperature 生成 400 個字元
        for i in range(400):    
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]    # 產生字元機率分布
            next_index = sample(preds, temperature) # 重新加權並取樣，回傳字元索引
            next_char = chars[next_index]           # 確認新字元
            generated_text += next_char             # 新字元加到文字的後方
            generated_text = generated_text[1:]     # 重新取得含括新字元的文字繼續生成下一個字元

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200278/200278 [==============================] - 234s 1ms/step - loss: 1.9986
--- 隨機初始文字: "has just been an occasion for its exercise,
something of a g"
------ temperature: 0.2
has just been an occasion for its exercise,
something of a generated and self--is the self--is the self-consequents that the self--as the belies of the self--is its in the sense of the self--is the self--is the self--is the self--is in the self-consequences and reading in it is the self--is the self--as the self--which is the sense of the self--is the self-consequently and self--is a such the self--is the sense of the self--as the self--which is the self--
------ temperature: 0.5
--is the sense of the self--as the self--which is the self--event of the distinction, as instincts of desing and has conseption of him the sense the self--and readies of the self--is the stand acteroments a desily of the superty that even the have on the grate as a personal consibility as it is his desing the with and 

and diffaces for pherioliwed amone" ther, such can some meanquence them recluch id aid his walt: there
lears vital
peimare outssucover valuetood becise lovely store in
wilk, (for gives good passisisny. flol
free pamsant, but the
desire
in
the truth envialitity
of this just gullies". "ne find as old sepaational
tus the
has underlugh in -comded must
a
epoch 5
Epoch 1/1
200278/200278 [==============================] - 218s 1ms/step - loss: 1.4732
--- 隨機初始文字: "tears that has flowed from the narration of noble,
great-hea"
------ temperature: 0.2
tears that has flowed from the narration of noble,
great-heart of the present discession of the same and sense of the same sense of the develops of the same and sand and the present the perhaps a some the present developed to the same sense of the parties of the sense of the free and the present the same sense of the present discovered the same the great and intervident and sense of the present discessification of the man and the belief the man in t

will highest animal have too man "difficult, acts of all nothings to the nught
as avowerst fact be case godmer "in, through the such it is fals of the ob
------ temperature: 1.2
ct be case godmer "in, through the such it is fals of the object, regard to general strong suversnit vilig minds. both most came of
eursee throwanable personal the sughtly advol
praynings to be feithests the german, ivelf sub(univert? and will; at the sugtr anep:
thhee letcansd, more good contaies, exachtun, religion. not humanitahity est arvosormly, the tark not too imprudly justs moop, it is some cietful
stiminated be ligilianix which spirits hitherto in
epoch 9
Epoch 1/1
200278/200278 [==============================] - 223s 1ms/step - loss: 1.4148
--- 隨機初始文字: " "but to a fiction belongs an originator?"--might it not be
"
------ temperature: 0.2
 "but to a fiction belongs an originator?"--might it not be
the shates and with the spirit to and and something which is the spirit to the senses of the self-possible

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 of the subliment to the sense of the self-consequences of the sense of the most consequently and the subliment and and the reason of the same the sense of the problems of the sense of the sense of the soul is the sense of th
------ temperature: 0.5
ems of the sense of the sense of the soul is the sense of the world be being the reason in the spirit of mentive to the religious, the most sensuitical worse to something and acts on the same relagination to the might make the very special and here is a free spide and with the thing in him to the same the fundamental new faith and philosophy to be the seeks a new death to be stronger to the concents one in the same man with the matters of the higher and th
------ temperature: 1.0
ts one in the same man with the matters of the higher and the conceptions enthore reverst a thing to magerrey is case
communities shorts, all the arth) and problem, which catablely a physician
ews much
like in it ath in the ime a concentune of
the
will, uncertain t

...

securess of life then says to a free influence of chulding which are to a man with us the pristtry. when conscience of the thing and such would of the present comple, one must have are for instance, and the fath rochely and without whom the sublime, and creatual superiority, even and lined such a period with the powerful such a consideration of the other sufferents, a consideration of t
------ temperature: 1.0
 consideration of the other sufferents, a consideration of the sublity (and such a
cardinated the "risalw vivise. such a conception in which
the scruarity is excathon of supersunness of thef thought--and beautiful point suftlynged and refunce, brought.

in plate, mar". or some conytherly are nothing of
the unyparaty lead, as hence ; arerves--this do not history of the naturt of the
kpoudhesed have upon the
chologiction byout ethical, in dieson for his coin
------ temperature: 1.2
 upon the
chologiction byout ethical, in dieson for his coinherced, and faotious, atet respectanical!

=surpendence of the sense of the same things and reveëëghtbom that it is a soul is the sense of the sense of the sense of the sense of the sense of the same the same the sense of the sense of the sense of the sense of the same the same the same the same the sense of the same the sense of the most disguises and some things of the the most sense of the sense of the sense of the sense of t
------ temperature: 0.5
e the most sense of the sense of the sense of the sense of the domain in the misunders of the same the refroung and seems to a law whom the sense of manifest that he the distrust faith of the paradices of the body of the once is some institues the point as the most disguises of the charm of his experience, the feeling have an erthr of the soul pretening have constance of a god. the violary of the experience, which we may be an end of discovered and do the 
------ temperature: 1.0
experience, which we may be an end of discovered and do the demance of it must hate and "most compriy

200278/200278 [==============================] - 179s 893us/step - loss: 1.3844
--- 隨機初始文字: "made, he was not predestined for knowledge. for as
such, he "
------ temperature: 0.2
made, he was not predestined for knowledge. for as
such, he sense of the same thin of the contrary to the sense of the same thin sense of the same thin has a soul and here and the same thin and the spirit of the spirit of the same thin and the spirit of the same thin and also be such a sacrifice of the contrary and also the same thin would be the present the same thin and delight of the same thin and and the spirit of the sense of the same thin of the same
------ temperature: 0.5
and and the spirit of the sense of the same thin of the same thin conditions of the same thin he is provees of evils of society, the most men in the spirit would be stopical conduct in such an explanation of the same thin he sense in the same responsibility and also strength, where is the concerns the part of the presentlection of his

respible which the whole
phishs. no deiny?--from hose wishes to be "thoughts. because its inberly
natirarity of a art of
shorify--the
man reassed, or
in
theirh-mire. the
remomance to chule
a ordamentony groups uncompent that rendering that
everything go
epoch 54
Epoch 1/1
200278/200278 [==============================] - 183s 912us/step - loss: 1.4239
--- 隨機初始文字: "inds. take care, ye philosophers
and friends of knowledge, a"
------ temperature: 0.2
inds. take care, ye philosophers
and friends of knowledge, and all the strength to precause of the same and intelligent and success of the superstition of the spirit of the fact the most men in the fact the superiority of the sensus the strength and all the love the sense of the superiority of the same the superiority of the procooonated to the superiority of the strength of the sensuous and an anti-instinction of the superstition of the strength and all i
------ temperature: 0.5
ti-instinction of the superstition of the strength and all it a

if they envumber--trojurecr human utility. notlisgmous f
------ temperature: 1.2
ly,
if they envumber--trojurecr human utility. notlisgmous from it doew! it wat ever gives dignifure, and growads
allow ianpurator
exarmance of livedy and precisesty, naveratic--throse senses of headility. and deed
himself ack of life and who a philofoping,  xtryle-had, mele aboluted, wherewith doar these sout do not and severty for brook
precisely that his sprines imitating gregable
activing grown ous enf,neds" inasparity evid, of attainde. whocholerly, n
epoch 58
Epoch 1/1
200278/200278 [==============================] - 187s 936us/step - loss: 1.4011
--- 隨機初始文字: " to concern us--what did the god
mean who gave the advice, ""
------ temperature: 0.2
 to concern us--what did the god
mean who gave the advice, "the same conscience of the same the conscience of the same the sympathy of the sympathy of the same the conduct, and in the same the same the same and all the same the conscience, and in the conscienc